### Quantum Fisher Information matrix

<img src="others\qfim.png">

I have coded QFIM function and test it on 1qubit case

$\partial_\theta|\psi\rangle$ can be calculated by on the below equation (from Le Bin Ho):

<img src="others/grad_psi.png"/>

In [1]:
import qiskit
import numpy as np
import qtm.base_qtm, qtm.constant, qtm.qtm_1qubit
import matplotlib.pyplot as plt
from types import FunctionType

In [2]:
def create_QFIM(psi: np.ndarray, grad_psi: np.ndarray):
    num_params = grad_psi.shape[0]
    F_elements = np.zeros(num_params, dtype = np.complex128)
    for i in range(num_params):
        print(F_elements[i])
        F_elements[i] = np.transpose(np.conjugate(psi)).dot(grad_psi[i])
    F = np.zeros([num_params, num_params])
    for i in range(0, num_params):
        for j in range(0, num_params):
            F[i, j] = 4*np.real(
                np.transpose(np.conjugate(grad_psi[i])).dot(grad_psi[j]) - 
                np.transpose(np.conjugate(F_elements[i]))*(F_elements[j]))
    return F

def grad_psi(
    qc: qiskit.QuantumCircuit, 
    create_circuit_func: FunctionType, 
    thetas: np.ndarray, r: float, s: float, **kwargs):
    gradient_psi = np.zeros([len(thetas), 2**qc.num_qubits], dtype = np.complex128)
    for i in range(0, len(thetas)):
        thetas_copy = thetas.copy()
        thetas_copy[i] += s
        qc1 = create_circuit_func(qc.copy(), thetas_copy, **kwargs)
        psi_qc = qiskit.quantum_info.Statevector.from_instruction(qc1).data
        gradient_psi[i] = r*(psi_qc)
    return gradient_psi

def qng(thetas: np.ndarray, psi: np.ndarray, grad_psi: np.ndarray, grad_loss: np.ndarray):
    F = create_QFIM(psi, grad_psi)
    inverse_F = np.linalg.pinv(F)
    thetas -= qtm.constant.learning_rate*np.dot(inverse_F, grad_loss)
    return thetas

def qng_adam(thetas: np.ndarray, 
    m: np.ndarray, v: np.ndarray, i: int, 
    psi: np.ndarray, grad_psi: np.ndarray, grad_loss: np.ndarray):
    F = create_QFIM(psi, grad_psi)
    inverse_F = np.linalg.pinv(F)
    grad = np.dot(inverse_F, grad_loss)
    thetas = qtm.base_qtm.adam(thetas, m, v, i, grad)
    return thetas

In [3]:
# Init quantum tomography 1 qubit
def a(qc, thetas):
    qc.h(0)
    qc.rz(thetas[0], 0)
    return qc
thetas = np.zeros((1,))
qc = qiskit.QuantumCircuit(1, 1)


grad_psi1 = grad_psi(qc.copy(), a, thetas, r = 1/2, s = np.pi)
print(grad_psi1)

qc_copy = a(qc.copy(), thetas)
psi = qiskit.quantum_info.Statevector.from_instruction(qc_copy).data
# Create gradient of loss function
# Quantum natural gradient
F = create_QFIM(psi, grad_psi1)
print(F)
    

[[2.16489014e-17-0.35355339j 2.16489014e-17+0.35355339j]]
0j
[[1.]]
